<a href="https://colab.research.google.com/github/ritesh-tiwary/ineuron/blob/main/python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Code

In [ ]:
streamLogger/config.py

import os

# Statement for enabling the development environment
DEBUG = True

# Define the application directory
BASE_DIR = os.path.abspath(os.path.dirname(__file__))

# Application threads. A common general assumption is
# using 2 per available processor cores - to handle
# incoming requests using one and performing background
# operations using the other.
THREADS_PER_PAGE = 2

# Enable protection against *Cross-site Request Forgery (CSRF)*
CSRF_ENABLED = True

# Use a secure, unique and absolutely secret key for
# signing the data.
CSRF_SESSION_KEY = "secret"



In [ ]:
streamLogger/run.py
# Run a test server.
from app import app
app.run(host='0.0.0.0', port=3000, debug=True)



In [ ]:
app/__init__.py

import json
from datetime import datetime, date
from flask import Flask, request, render_template

# Define the WSGI application object
app = Flask(__name__)

# Configurations
app.config.from_object('config')

d = date.today().strftime("%d-%b-%Y")
y = date.today().strftime("%Y")


@app.route('/')
def index():
    try:
        with open("app/logs/".__add__(y).__add__("/").__add__(d).__add__(".json")) as f:
            data = json.load(f)
    except FileNotFoundError:
        data = []
    return render_template('index.html', data=data, utc_dt=datetime.utcnow())


@app.route("/logs", methods=["post"])
def logs():
    try:
        logs = []
        log = request.data
        with open("app/logs/".__add__(y).__add__("/").__add__(d).__add__(".json")) as f:
            logs = json.load(f)
        logs.append(json.loads(log))
        with open("app/logs/".__add__(y).__add__("/").__add__(d).__add__(".json"), "w") as f:
            json.dump(logs, f, indent=4, separators=(',', ': '))
    except FileNotFoundError:
        logs.append(json.loads(log))
        with open("app/logs/".__add__(y).__add__("/").__add__(d).__add__(".json"), "w") as f:
            json.dump(logs, f, indent=4, separators=(',', ': '))
    return "OK", 200


# Sample HTTP error handling
@app.errorhandler(404)
def not_found(error):
    return render_template('404.html'), 404



In [ ]:
streamLogger/streamLogger.py

import json
import logging
import requests

class RequestsHandler(logging.Handler):
    def emit(self, record):
        log = self.format(record)
        # response = requests.post('http://localhost:3000/logs', headers={"Content-type": "application/json"}, data=log)
        # print("Status Code", response.status_code)
        print(log)
        return requests.post('http://localhost:3000/logs',
                             headers={"Content-type": "application/json"},
                             data=log).content


class FormatterLogger(logging.Formatter):
    def __init__(self, task_name=None):
        super(FormatterLogger, self).__init__()

    def format(self, record):
        data = {'@message': record.msg,
                '@funcName': record.funcName,
                '@exc_text': record.exc_text,
                '@lineno': 0 if record.exc_text is None else record.lineno,
                '@exc_info': record.exc_info,
                }

        return json.dumps(data)


def custom_logger(name):
    logger = logging.getLogger(name)
    custom_handler = RequestsHandler()
    formatter = FormatterLogger(logger)
    custom_handler.setFormatter(formatter)
    logger.addHandler(custom_handler)
    return logger


logger = custom_logger("ECA")
logger.exception("ECA - Job Failed")
logger.exception("ECA - Job Completed Successfully")


In [ ]:
streamLogger/app/templates/index.html

{% extends 'base.html' %}

{% block content %}
<!--    <h1>{% block title %} Index {% endblock %}</h1>-->
<!--    <h1>Hello World!</h1>-->
<!--    <h2>Welcome to FlaskApp!</h2>-->
    <h3 style="margin-left: 20px;">UTC Time: {{ utc_dt }}</h3>
    <div style="width: 100%; margin: auto">
        <div style="display: flex; padding: 10px; background-color: #e7f9ff; margin: 20px">
            <div style="width: 25%; display: inline-block; font-weight: bold;><p style="font-size: 100px"> Message </p></div>
            <div style="width: 20%; display: inline-block; font-weight: bold;><p style="font-size: 32px"> Method Name </p></div>
            <div style="width: 20%; display: inline-block; font-weight: bold;><p style="font-size: 32px"> Exception Text </p></div>
            <div style="width: 10%; display: inline-block; font-weight: bold;><p style="font-size: 32px"> Line No </p></div>
            <div style="width: 25%; display: inline-block; font-weight: bold;><p style="font-size: 32px"> Traceback </p></div>
        </div>
        {% for d in data %}
            {% if loop.index % 2 == 0 %}
                    {% set bg_color = '#e6f9ff' %}
                {% else %}
                    {% set bg_color = '#eee' %}
                {% endif %}

                <div style="display: flex; padding: 10px; background-color: {{ bg_color }}; margin: 20px">
                    <div style="width: 25%; display: inline-block;><p style="font-size: 24px"> {{ d["@message"] }} </p></div>
                    <div style="width: 20%; display: inline-block;><p style="font-size: 24px"> {{ d["@funcName"] }} </p></div>
                    <div style="width: 20%; display: inline-block;><p style="font-size: 24px"> {{ d["@exc_text"] }} </p></div>
                    <div style="width: 10%; display: inline-block;><p style="font-size: 24px"> {{ d["@lineno"] }}   </p></div>
                    <div style="width: 25%; display: inline-block;><p style="font-size: 24px"> {{ d["@exc_info"] }} </p></div>
                </div>
        {% endfor %}
    </div>
{% endblock %}



In [ ]:
streamLogger/app/templates/base.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{% block title %} {% endblock %} - FlaskApp</title>
    <style>
        nav a {
            color: #d64161;
            font-size: 3em;
            margin-left: 20px;
            text-decoration: none;
        }
    </style>
</head>
<body>
    <nav>
        <a href="#">C D R&nbsp;&nbsp;J O B&nbsp;&nbsp;D A S H B O A R D</a>
    </nav>
    <hr>
    <div class="content">
        {% block content %} {% endblock %}
    </div>
</body>
</html>

In [ ]:
streamLogger/app/templates/404.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>404</title>
</head>
<body>

</body>
</html>

In [ ]:
streamLogger/app/logs/2022/23-Jun-2022.json

- https://onboardbase.com/blog/aes-encryption-decryption
- https://www.quickprogrammingtips.com/python/aes-256-encryption-and-decryption-in-python.html
- https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher
- https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/

In [ ]:
# !pip install pycrypto
# AES/AESCipher.py

from hashlib import sha256
from Crypto import Random
from Crypto.Cipher import AES
from base64 import b64encode, b64decode
from Crypto.Random import get_random_bytes

class AESCipher:
    def __init__(self, key):
        self.aes_mode = AES.MODE_CBC
        self.block_size = AES.block_size	
        self.key = sha256(key.encode()).digest()

    def encrypt(self, plain_text):
        plain_text = self.__pad(plain_text)
        iv = Random.new().read(self.block_size)
        cipher = AES.new(self.key, self.aes_mode, iv)
        encrypted_text = cipher.encrypt(plain_text.encode())
        return b64encode(iv + encrypted_text).decode("utf-8")

    def decrypt(self, encrypted_text):
        encrypted_text = b64decode(encrypted_text)
        iv = encrypted_text[:self.block_size]
        cipher = AES.new(self.key, self.aes_mode, iv)
        plain_text = cipher.decrypt(encrypted_text[self.block_size:]).decode("utf-8")
        return self.__unpad(plain_text)

    def __pad(self, plain_text):
        number_of_bytes_to_pad = self.block_size - len(plain_text) % self.block_size
        ascii_string = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_string
        padded_plain_text = plain_text + padding_str
        return padded_plain_text

    @staticmethod
    def __unpad(plain_text):
        last_character = plain_text[len(plain_text) - 1:]
        return plain_text[:-ord(last_character)]

if __name__ == "__main__":
	key = "m@4989R$8h"
	data = "Plain text"
	encrypted_data = AESCipher(key).encrypt(data)
	decrypted_data = AESCipher(key).decrypt(encrypted_data)

	print(encrypted_data)
	print(decrypted_data)


L4ZquH1p4WAh+hKo2w+AAlI1Ulhx1/PKcnImmP1SY30=
Plain text


In [ ]:
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]

In [ ]:
kdf = PBKDF2HMAC(
  algorithm=hashes.SHA256(),
  length=32,
  salt=urandom(16),
  iterations=390000,
  )

key = kdf.derive(b"An-encryption-key")
key

b'l\xa3f\xea\xb0v\x9f\x10\x1e\xb2{\x9e:>\xe0c/\xe5\x11\xb2\x15%\xea\xb1\x10@\xd02\xeeI\x03\xed'

In [ ]:
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=urandom(16),
    iterations=390000,
)

kdf.verify(b"An-encryption-key", b'l\xa3f\xea\xb0v\x9f\x10\x1e\xb2{\x9e:>\xe0c/\xe5\x11\xb2\x15%\xea\xb1\x10@\xd02\xeeI\x03\xed')

InvalidKey: ignored

In [ ]:
salt = urandom(16)
# derive
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=390000,
)
key = kdf.derive(b"my great password")
# verify
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=390000,
)

kdf.verify(b"my great password", key)

In [ ]:
# !pip install cryptography
import os
from os import urandom
from Crypto.Protocol.KDF import PBKDF2
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

kdf = PBKDF2HMAC(
  algorithm=hashes.SHA256(),
  length=32,
  salt=urandom(64),
  iterations=390000,
  )

key = kdf.derive(b"An-encryption-key")
iv = urandom(16)
# iv = get_random_bytes(16)
cipher = Cipher(algorithms.AES256(key), modes.CBC(iv))
encryptor = cipher.encryptor()
encrypted_data = encryptor.update(b"a secret message") + encryptor.finalize()
encrypted_data

b'q\xfa\xf6\xec\xe3\x1eB\xd2\xc1v\xd2-<\x1b5S'

In [ ]:
encrypted_data = b'\xce>\xb3@\x05\x1azI\x16\x9d(2\x8d\xa6\xba\x8b'
decryptor = cipher.decryptor()
decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()
decrypted_data.decode("utf-8")

'a secret message'

In [ ]:
help(algorithms)

In [ ]:
import os
iv = os.urandom(16)
iv

b'5<<\x0c&em\x00\x81\xb4\x13\x1b\xd1\xb7\x06\x89'